In [1]:
import os
import pickle
from pprint import pprint
import datetime
import numpy as np
from matplotlib import pyplot as plt
import torch
from structure_utils import *

dirname = 'OneDatum2'

losses = []
with open(dirname+'/loss.txt', 'r') as f:
    for iteration in f:
        losses.append([float(term) for term in iteration.split()])
losses = np.array(losses)
# print(losses)
N_iters = losses.shape[0]

# Plot total loss
plt.figure(figsize=(20, 15))
plt.plot(range(N_iters), np.sum(losses, axis=1), label='Total')
plt.title('Loss')
plt.xlabel('Iteration')
plt.ylabel('Loss')
# plt.show()

# Plot respective losses
colors = ['r', 'g', 'y', 'k']
for i in range(4):
    plt.plot(range(N_iters), losses[:, i], colors[i], label='Loss' + str(i+1))

plt.legend(loc='upper right')
plt.show()

# Plot loss1 separately
plt.figure()
plt.title('Loss1')
plt.plot(range(N_iters), losses[:, 0])
plt.show()


<Figure size 2000x1500 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [2]:
# Display hyperparameters and run information

params = pickle.load(open(dirname+'/hyperparameters.pkl', 'rb'))
pprint(params)

runtime = pickle.load(open(dirname+'/runtime.pkl', 'rb'))
print('\nRuntime: ' + str(datetime.timedelta(seconds=runtime)))

{'epochs': 5000,
 'lambdas': (20, 1, 0, 20.0),
 'learning_rate': 1e-05,
 'loss2_margin': 1,
 'multiplier': 0.001,
 'n_processes': 1,
 'nb_data': 1,
 'scoring_hidden_size': 128}

Runtime: 12:50:33.085488


In [3]:
# Display weight magnitudes

# epochs = params['epochs']
# magnitudes = np.zeros(epochs)
# scoring = np.zeros(epochs)
# for i in range(epochs):
#     weights = torch.load(dirname + '/epoch_' + str(i + 1) + '.pt')
#     for weight in weights:
#         magnitudes[i] += weights[weight].norm()
#         if 'scoring' in weight:
#             scoring[i] += weights[weight].norm()

# plt.figure()
# plt.plot(range(epochs), magnitudes, 'g')
# plt.title('Norm of weights')
# plt.xlabel('Epoch')
# plt.show()

# plt.figure()
# plt.plot(range(epochs), scoring, 'r')
# plt.title('Norm of scoring')
# plt.xlabel('Epoch')
# plt.show()

In [4]:
# Show 5 different structures from training

with open(dirname+'/structure.txt') as f:
    lines = f.readlines()
    structures = []
    for line in lines:
        if line != '\n':
            structures.append(line)
            
    spacing = 1000
    for i in range(0, N_iters, spacing):
        print('Iteration', i)
        print(structures[i])
        print()
    
    print('Final structure')
    print(structures[-1])

Iteration 0
[['x', 'h', 1, 'add', 'minus', 'G0'], ['x', 'h', 1, 'add', 'minus', 'G1'], ['x', 'h', 1, 'add', 'minus', 'G2'], ['x', 'h', 1, 'add', 'minus', 'G3'], ['x', 'h', 1, 'add', 'minus', 'G4'], ['G0', 'G1', 1, 'add', 'minus', 'G5'], ['G5', 'G2', 1, 'add', 'minus', 'G6'], ['G6', 'G3', 1, 'add', 'minus', 'G7'], ['G7', 'G4', 1, 'add', 'minus', 'G8']]


Iteration 1000
[['x', 'h', 2, 'add', 'minus', 'G0'], ['x', 'h', 2, 'add', 'minus', 'G1'], ['x', 'h', 2, 'add', 'minus', 'G2'], ['x', 'h', 2, 'add', 'minus', 'G3'], ['x', 'h', 2, 'add', 'minus', 'G4'], ['G0', 'G1', 0, 'mul', 'minus', 'G5'], ['G5', 'G2', 0, 'mul', 'minus', 'G6'], ['G6', 'G3', 0, 'mul', 'minus', 'G7'], ['G7', 'G4', 0, 'mul', 'minus', 'G8']]


Iteration 2000
[['x', 'h', 2, 'add', 'minus', 'G0'], ['0', 'G0', 2, 'add', 'minus', 'G1'], ['x', 'h', 2, 'add', 'minus', 'G2'], ['0', 'G2', 2, 'add', 'minus', 'G3'], ['x', 'h', 2, 'add', 'minus', 'G4'], ['0', 'G4', 2, 'add', 'minus', 'G5'], ['0', 'G1', 1, 'mul', 'minus', 'G6'], ['G6',

In [10]:
# Show GRU structures
import ast

best = None
fewest_differences = np.Inf
best_iter = 0
for i, structure in enumerate(structures):
    structure = ast.literal_eval(structure)
    if structure_is_gru(structure):
        diffs = n_differences(structure, GRU_STRUCTURE)
        if diffs < fewest_differences:
            best = structure
            fewest_differences = diffs
            best_iter = i

            
if best is None:
    print('No GRU structures found.')
else:
    print('Best structure (%d differences from GRU) achieved on iteration %d' % (fewest_differences, best_iter))
    pprint(best)

Best structure (11 differences from GRU) achieved on iteration 405
[['x', 'h', 1, 'add', 'minus', 'G0'],
 ['x', 'G0', 1, 'add', 'minus', 'G1'],
 ['x', 'h', 1, 'add', 'minus', 'G2'],
 ['x', 'G2', 1, 'add', 'minus', 'G3'],
 ['x', 'h', 1, 'add', 'minus', 'G4'],
 ['x', 'G4', 1, 'add', 'minus', 'G5'],
 ['0', 'G1', 1, 'add', 'minus', 'G6'],
 ['G6', 'G3', 1, 'add', 'minus', 'G7'],
 ['G7', 'G5', 1, 'add', 'minus', 'G8']]
